In [1]:
import pandas as pd
import os
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import plotly.express as px
import plotly
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
print('Inicializado Rotina json_collect.py.')
os.chdir(sys.path[0])
# os.chdir(os.path.join(os.getcwd(),"json_collect.py"))                  # Modifica o diretório de trabalho, para os caminhos relativos funcionar. (https://stackoverflow.com/questions/1432924/python-change-the-scripts-working-directory-to-the-scripts-own-directory)
# print(os.getcwd())
%run ./json_collect.py                      

Inicializado Rotina json_collect.py.
json_collect executed


In [3]:
# sites: https://dadosabertos.bcb.gov.br/dataset/expectativas-mercado
# Sintase: https://dadosabertos.bcb.gov.br/dataset/expectativas-mercado/resource/d420a704-75a7-4f45-8f4b-0fca813c70f0
# documentação: https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/documentacao

In [3]:
# Relações datas das reunições inseridas manualmente quando não tem previsão
df_selic_data=pd.read_csv('./input/selic_datas.csv',index_col='Reuniao')
# Dados baixados pelo .json
df_selic=pd.read_csv('./output/selic_data_df.csv',index_col='Reuniao')
df_selic=df_selic.drop(columns=['Unnamed: 0','Indicador'])
df_selic['Data']=df_selic.apply(lambda x: datetime.datetime.strptime(x.Data,'%Y-%m-%d').date(),axis=1)

# Relaciona a reunião com sua Data.
df_selic['data_reuniao']=''
for i in range(0, len(df_selic)):
    busca_data=''
    try:
        busca_data=df_selic_data.loc[df_selic.index[i],'data_reuniao']
        busca_data=datetime.datetime.strptime(busca_data,'%Y-%m-%d').date()
        df_selic.loc[df_selic.index[i],'data_reuniao']=busca_data

    except:
        #### será definida um expectativa para a data.
        # busca_data=df_selic_data.loc[df_selic.index[i],'data_reuniao']
        # df_selic.loc[df_selic.index[i],'data_reuniao']=busca_data
        continue

# df_selic['data_reuniao']=df_selic.apply(lambda x: datetime.datetime.strptime(x.data_reuniao,'%Y-%m-%d').date(),axis=1)
# df_selic=df_selic[df_selic['data_reuniao']!='']
df_selic=df_selic.sort_values(by=['Data','data_reuniao','baseCalculo'],ascending=[False,True,False])

'''Base de cálculo para as estatísticas
 baseada no prazo de validade das expectativas
 informadas pelas instituições informantes:
  - 0: uso das expectativas mais recentes informadas
   pelas instituições participantes a partir do
   30º dia anterior à data de cálculo das estatísticas 
  - 1: uso das expectativas mais recentes informadas
   pelas instituições participantes a partir do
    4º dia útil anterior à data de cálculo das estatísticas'''

df_selic.head(20)

,Data,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo,data_reuniao
Reuniao,,,,,,,,,
R1/2023,2023-01-20,13.75,13.75,0.00,13.75,13.75,79,1,2023-02-01
R1/2023,2023-01-20,13.75,13.75,0.00,13.75,13.75,127,0,2023-02-01
R2/2023,2023-01-20,13.75,13.75,0.00,13.75,13.75,79,1,2023-03-22
R2/2023,2023-01-20,13.75,13.75,0.00,13.75,13.75,127,0,2023-03-22
R3/2023,2023-01-20,13.73,13.75,0.10,13.00,13.75,79,1,2023-05-03
R3/2023,2023-01-20,13.74,13.75,0.08,13.00,13.75,127,0,2023-05-03
R4/2023,2023-01-20,13.69,13.75,0.20,12.50,13.75,79,1,2023-06-21
R4/2023,2023-01-20,13.69,13.75,0.18,12.50,13.75,127,0,2023-06-21
R5/2023,2023-01-20,13.56,13.75,0.36,12.00,13.75,79,1,2023-08-02


In [4]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_selic,x='data_reuniao',y='Mediana', color='Data')
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
fig.show()

In [5]:
# Informada à partir do 4º dia útil anterior à data de cálculo da expectativa.
df_selic=df_selic[df_selic['baseCalculo']==1]           
# Expectativa mais atual
df_selic_ultima=df_selic[df_selic['Data']==df_selic['Data'].max()]
# df_selic_ultima['mediana_mensal']=df_selic_ultima.apply(lambda x: (1+(x.Mediana/100))**(1/12)-1,axis=1)

In [6]:
# Insere coluna com números de dias no período (entre duas datas)
df_selic_ultima.loc[df_selic_ultima.index[0],'periodo']=(df_selic_ultima.loc[df_selic_ultima.index[0],'data_reuniao']-datetime.datetime.today().date()).days
# Define número índice de ref. 100 na data de hoje.
selic_atual=''
while type(selic_atual)!=float:
    try:
        selic_atual=float(input('Qual é a SELIC atual?(\d\d.d\d\)'))
    except:
        continue

C:\Users\bruno.bfn\AppData\Local\Temp\ipykernel_1868\1858555858.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
# Cria num_indice inicial atribuindo 100 para a data de hoje
## Pode acontecer de demora para atualização do json do bc, gerando o primeiro período negativo, mas não tem problema
df_selic_ultima.loc[df_selic_ultima.index[0],'num_indice']=100*(1+selic_atual/100)**(df_selic_ultima.loc[df_selic_ultima.index[0],'periodo']/365)

# num_indice_f=num_indice_i*(1+taxa)**(periodo)
for i in range(1,len(df_selic_ultima)):
    df_selic_ultima.loc[df_selic_ultima.index[i],'periodo']=(df_selic_ultima.loc[df_selic_ultima.index[i],'data_reuniao']-df_selic_ultima.loc[df_selic_ultima.index[i-1],'data_reuniao']).days
    df_selic_ultima.loc[df_selic_ultima.index[i],'num_indice']=df_selic_ultima.loc[df_selic_ultima.index[i-1],'num_indice']*(1+df_selic_ultima.loc[df_selic_ultima.index[i-1],'Mediana']/100)**(df_selic_ultima.loc[df_selic_ultima.index[i],'periodo']/365)
df_selic_ultima.to_csv('./output/selic_ultima_expectativa.csv',float_format='%.2f')


C:\Users\bruno.bfn\AppData\Local\Temp\ipykernel_1868\2648126627.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
df_selic_ultima

,Data,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo,data_reuniao,periodo,num_indice
Reuniao,,,,,,,,,,,
R1/2023,2023-01-20,13.75,13.75,0.00,13.75,13.75,79,1,2023-02-01,4.00,100.14
R2/2023,2023-01-20,13.75,13.75,0.00,13.75,13.75,79,1,2023-03-22,49.00,101.89
R3/2023,2023-01-20,13.73,13.75,0.10,13.00,13.75,79,1,2023-05-03,42.00,103.41
R4/2023,2023-01-20,13.69,13.75,0.20,12.50,13.75,79,1,2023-06-21,49.00,105.21
R5/2023,2023-01-20,13.56,13.75,0.36,12.00,13.75,79,1,2023-08-02,42.00,106.79
R6/2023,2023-01-20,13.30,13.50,0.57,11.50,13.75,79,1,2023-09-20,49.00,108.65
R7/2023,2023-01-20,12.91,13.00,0.74,11.00,13.75,79,1,2023-11-01,42.00,110.24
R8/2023,2023-01-20,12.52,12.50,0.93,10.25,13.75,79,1,2023-12-13,42.00,111.80
R1/2024,2023-01-20,12.10,12.00,1.01,9.50,13.75,74,1,2024-01-31,49.00,113.59


In [9]:
#### Função que calcula a SELIC do período entre data atual e data final
def calc_selic_periodo(df,data_f):
    # DF é um dataframe que contenha a culuna com num_indice
    
    # Utiliza a taxa digitada real vigente
    if data_f<=df.loc[df.index[0],'data_reuniao']:
        n_dias=(data_f-datetime.datetime.today().date()).days
        return 100*(1+selic_atual/100)**(n_dias/365)
    
    # Utiliza o número índice anterior + variação dos dias dentro do período
    else:
        # Último número índice
        df=df[df['data_reuniao']<=data_f]
        n_indice=df.loc[df.index[-1],'num_indice']
        taxa=df.loc[df.index[-1],'Mediana']
        n_dias=(data_f-df.loc[df.index[-1],'data_reuniao']).days
        return n_indice*(1+taxa/100)**(n_dias/365)


In [10]:
calc_selic_periodo(df_selic_ultima,datetime.datetime(2022,10,25).date())

96.70241174475082

In [11]:
df_monthly=pd.read_csv('./output/monthly_data_df.csv')
df_monthly=df_monthly.drop(columns=['Unnamed: 0'])

# Parses
df_monthly['Data']=df_monthly.apply(lambda x: datetime.datetime.strptime(x.Data,'%Y-%m-%d').date(),axis=1)

df_monthly['DataReferencia']=df_monthly.apply(lambda x: datetime.datetime.strptime(x.DataReferencia,'%m/%Y').date(),axis=1)
delta = relativedelta(months=1)
delta_d = timedelta(days=1)
df_monthly['DataReferencia']=df_monthly.apply(lambda x: x.DataReferencia+delta-delta_d,axis=1)

df_monthly=df_monthly.sort_values(by=['Data','DataReferencia','numeroRespondentes'],ascending=[False,True,False])
# df_monthly=df_selic[df_selic['baseCalculo']==1]           # não sei qual é a diferença da base de cálculo.
df_monthly.head(20)

,Indicador,Data,DataReferencia,Media,Mediana,Minimo,Maximo,numeroRespondentes
419,Taxa de desocupação,2023-01-20,2022-12-31,8.09,8.00,7.50,8.91,58
418,Taxa de desocupação,2023-01-20,2022-12-31,8.08,8.00,7.74,8.91,40
403,IPCA,2023-01-20,2023-01-31,0.51,0.50,0.15,1.23,141
405,Câmbio,2023-01-20,2023-01-31,5.23,5.23,5.00,5.50,108
402,IPCA,2023-01-20,2023-01-31,0.51,0.50,0.37,0.69,98
417,IPCA Administrados,2023-01-20,2023-01-31,0.60,0.56,0.02,3.51,84
409,IPCA Livres,2023-01-20,2023-01-31,0.47,0.47,0.21,0.69,79
404,Câmbio,2023-01-20,2023-01-31,5.22,5.22,5.00,5.40,76
411,IPCA Serviços,2023-01-20,2023-01-31,0.35,0.33,0.01,0.64,72
413,IPCA Bens industrializados,2023-01-20,2023-01-31,0.44,0.43,0.00,0.95,72


In [12]:
df_monthly['Indicador'].unique()

array(['Taxa de desocupação', 'IPCA', 'Câmbio', 'IPCA Administrados',
       'IPCA Livres', 'IPCA Serviços', 'IPCA Bens industrializados',
       'IPCA Alimentação no domicílio', 'IGP-M'], dtype=object)

In [13]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='IPCA'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='IGP-M'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/IGPM_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='Câmbio'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/cambio_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='Taxa de desocupação'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/tx_desocupacao_forcast.html')
# fig.show()

'./output/tx_desocupacao_forcast.html'

In [14]:
# Expectativa mais atual para IPCA
df_ipca=df_monthly[df_monthly['Indicador']=='IPCA']

data_previsao=df_ipca['Data'].max()
df_ipca_ultima=df_ipca[df_ipca['Data']==data_previsao]
df_ipca_ultima=df_ipca_ultima.groupby(['DataReferencia']).mean() # adotado média 

df_ipca_ultima['cumsum_mediana'] = df_ipca_ultima['Mediana'].cumsum()
#cumsum()-> soma acumulada de uma lista ou array termo a termo
#groupby -> delimita em grupos separados para análise
# df['Adjusted Quantity'] = df.groupby('ação_ref')['Adjusted Quantity'].cumsum()

In [15]:
# Insere coluna com números de dias no período (entre duas datas)
df_ipca_ultima.loc[df_ipca_ultima.index[0],'periodo']=(df_ipca_ultima.index[0]-datetime.datetime.today().date()).days

# Define número índice de ref. 100 na data de hoje.
df_ipca_ultima.loc[df_ipca_ultima.index[0],'num_indice']=100*(1+df_ipca_ultima.loc[df_ipca_ultima.index[0],'Mediana']/100)**(df_ipca_ultima.loc[df_ipca_ultima.index[0],'periodo']/30)
for i in range(1,len(df_ipca_ultima)):
    df_ipca_ultima.loc[df_ipca_ultima.index[i],'periodo']=(df_ipca_ultima.index[i]-df_ipca_ultima.index[i-1]).days
    df_ipca_ultima.loc[df_ipca_ultima.index[i],'num_indice']=df_ipca_ultima.loc[df_ipca_ultima.index[i-1],'num_indice']*(1+df_ipca_ultima.loc[df_ipca_ultima.index[i],'Mediana']/100)**(df_ipca_ultima.loc[df_ipca_ultima.index[i],'periodo']/30)
df_ipca_ultima.to_csv('./output/ipca_ultima_expectativa.csv',float_format='%.4f')
df_ipca_ultima

,Media,Mediana,Minimo,Maximo,numeroRespondentes,cumsum_mediana,periodo,num_indice
DataReferencia,,,,,,,,
2023-01-31,0.51,0.50,0.26,0.96,119.50,0.50,3.00,100.05
2023-02-28,0.70,0.70,0.40,0.98,119.50,1.20,28.00,100.70
2023-03-31,0.63,0.57,0.10,1.30,119.00,1.77,31.00,101.30
2023-04-30,0.57,0.54,0.20,1.23,117.00,2.31,30.00,101.85
2023-05-31,0.37,0.37,0.11,0.85,117.00,2.68,31.00,102.24
2023-06-30,0.35,0.35,0.12,0.70,117.00,3.03,30.00,102.60
2023-07-31,0.30,0.30,-0.07,0.60,117.00,3.33,31.00,102.91
2023-08-31,0.24,0.24,0.03,0.62,117.00,3.57,31.00,103.17
2023-09-30,0.32,0.32,0.04,0.51,117.00,3.89,30.00,103.50


In [16]:
# from matplotlib.pyplot import figure
# figure(figsize=(10, 5), dpi=120)
# plt.plot(df_ipca_ultima.index, df_ipca_ultima['Mediana'], label=f'ipca_expectaiva_{data_previsao}')
# plt.xticks(rotation=90)
# plt.legend()
# plt.show()


In [17]:
#### Função que calcula a IPCA do período entre data hoje e data final
def calc_ipca_periodo(df,data_f):
    # DF é um dataframe que contenha a coluna com num_indice
    
    # Utiliza a taxa digitada real vigente
    if data_f<=df.index[0]:
        # print('primeiro mês')
        n_dias=(data_f-datetime.datetime.today().date()).days
        # print(n_dias)
        return 100*(1+df.loc[df.index[0],'Mediana']/100)**(n_dias/30)
    
    # Não calcula num_indice para o último termo, pois já está tabelado. 
    elif data_f==df.index[-1]:
        return df.loc[df.index[-1],'num_indice']

    # Utiliza o número índice anterior + variação dos dias dentro do período
    else:
        # Último número índice
        df_anterior=df[df.index<=data_f]
        # print(df_anterior)
        n_indice=df_anterior.loc[df_anterior.index[-1],'num_indice']
        #busca o 1º termo depois do corte da df_anterior
        taxa=df.loc[df.index[len(df_anterior)],'Mediana']    
        # print(f'taxa:{taxa}')
        n_dias=(data_f-df_anterior.index[-1]).days
        # print(f'n_dias:{n_dias}')
        return n_indice*(1+taxa/100)**(n_dias/30)

In [18]:
calc_ipca_periodo(df_ipca_ultima,datetime.datetime(2022,11,30).date())

99.02391180458193

In [19]:
df_ipca_ultima

,Media,Mediana,Minimo,Maximo,numeroRespondentes,cumsum_mediana,periodo,num_indice
DataReferencia,,,,,,,,
2023-01-31,0.51,0.50,0.26,0.96,119.50,0.50,3.00,100.05
2023-02-28,0.70,0.70,0.40,0.98,119.50,1.20,28.00,100.70
2023-03-31,0.63,0.57,0.10,1.30,119.00,1.77,31.00,101.30
2023-04-30,0.57,0.54,0.20,1.23,117.00,2.31,30.00,101.85
2023-05-31,0.37,0.37,0.11,0.85,117.00,2.68,31.00,102.24
2023-06-30,0.35,0.35,0.12,0.70,117.00,3.03,30.00,102.60
2023-07-31,0.30,0.30,-0.07,0.60,117.00,3.33,31.00,102.91
2023-08-31,0.24,0.24,0.03,0.62,117.00,3.57,31.00,103.17
2023-09-30,0.32,0.32,0.04,0.51,117.00,3.89,30.00,103.50


In [20]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_ipca_ultima,x=df_ipca_ultima.index,y=['cumsum_mediana'])
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/tx_desocupacao_forcast.html')
fig.show()

In [21]:
# Relação de datas até o mínimo entre os dataframes, para evitar erros
lista_datas_todos_dias=pd.date_range(start=datetime.datetime.today().date(),end=min(df_selic_ultima['data_reuniao'].max(),df_ipca_ultima.index.max()))
df_juros_real=pd.DataFrame(index=lista_datas_todos_dias,columns=['num_indice_selic','num_indice_ipca'],dtype='float64')
for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],'num_indice_selic']=calc_selic_periodo(df_selic_ultima,df_juros_real.index[i].date())
    df_juros_real.loc[df_juros_real.index[i],'num_indice_ipca']=calc_ipca_periodo(df_ipca_ultima,df_juros_real.index[i].date())
df_juros_real

,num_indice_selic,num_indice_ipca
2023-01-28,100.00,100.00
2023-01-29,100.04,100.02
2023-01-30,100.07,100.03
2023-01-31,100.11,100.05
2023-02-01,100.14,100.07
...,...,...
2024-12-11,124.13,108.66
2024-12-12,124.16,108.68
2024-12-13,124.19,108.69
2024-12-14,124.22,108.71


In [22]:
df_ipca_ultima

,Media,Mediana,Minimo,Maximo,numeroRespondentes,cumsum_mediana,periodo,num_indice
DataReferencia,,,,,,,,
2023-01-31,0.51,0.50,0.26,0.96,119.50,0.50,3.00,100.05
2023-02-28,0.70,0.70,0.40,0.98,119.50,1.20,28.00,100.70
2023-03-31,0.63,0.57,0.10,1.30,119.00,1.77,31.00,101.30
2023-04-30,0.57,0.54,0.20,1.23,117.00,2.31,30.00,101.85
2023-05-31,0.37,0.37,0.11,0.85,117.00,2.68,31.00,102.24
2023-06-30,0.35,0.35,0.12,0.70,117.00,3.03,30.00,102.60
2023-07-31,0.30,0.30,-0.07,0.60,117.00,3.33,31.00,102.91
2023-08-31,0.24,0.24,0.03,0.62,117.00,3.57,31.00,103.17
2023-09-30,0.32,0.32,0.04,0.51,117.00,3.89,30.00,103.50


In [23]:
# Não utilizada
def area(tipo,date):
    #tipo={selic,ipca}
    area=0
    df=df_juros_real[df_juros_real.index<=date]
    if (tipo=='selic' or tipo=='ipca'):
        for i in range(0,len(df)):
            area=area+1*df.loc[df.index[i],f'num_indice_{tipo}']
                
        return area
    else:
        print('Erro na escolha do tipo (selic ou ipca)')
    return 0

In [24]:
# df_juros_real['area_selic']=0
# for i in range(0,len(df_juros_real)):
#     # print(area('selic',df_juros_real.index[i]))
#     df_juros_real.loc[df_juros_real.index[i], 'area_selic']=area('selic',df_juros_real.index[i])
#     df_juros_real.loc[df_juros_real.index[i], 'area_ipca']=area('ipca',df_juros_real.index[i])

# df_juros_real

In [25]:
df_juros_real['dif_num_indice']=df_juros_real.apply(lambda x: x.num_indice_selic-x.num_indice_ipca+100,axis=1)
# df_juros_real['dif_area']=df_juros_real.apply(lambda x: x.area_selic-x.area_ipca,axis=1)
# df_juros_real['dif_area_por_dia']=df_juros_real.apply(lambda x: x.dif_area/(x.index-))
df_juros_real.to_csv('./output/num_indice_selic_ipca_diferenca.csv',float_format='%.2f')
df_juros_real


,num_indice_selic,num_indice_ipca,dif_num_indice
2023-01-28,100.00,100.00,100.00
2023-01-29,100.04,100.02,100.02
2023-01-30,100.07,100.03,100.04
2023-01-31,100.11,100.05,100.06
2023-02-01,100.14,100.07,100.07
...,...,...,...
2024-12-11,124.13,108.66,115.47
2024-12-12,124.16,108.68,115.48
2024-12-13,124.19,108.69,115.49
2024-12-14,124.22,108.71,115.51


In [26]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_juros_real,x=df_juros_real.index,y=['num_indice_selic','num_indice_ipca','dif_num_indice'])
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
fig.show()

In [27]:
from time import strptime
from pyxirr import xirr
# calculo da IRR
def cal_irr(x,date,tipo):                #DF refer^rencia e a data de análise

    if date==x.index[0]:
        return (0,0,0,0)
    
    dates=[x.index[0],date]
    
    # tipo pode assumir ('selic' ou 'pre')
    if tipo=='selic':
        amounts=[-x.loc[x.index[0],'dif_num_indice'],x.loc[date,'dif_num_indice']]
    elif tipo=='pre_1':
        amounts=[-x.loc[x.index[0],'dif_num_indice_pre_1'],x.loc[date,'dif_num_indice_pre_1']]
    elif tipo=='pre_2':
        amounts=[-x.loc[x.index[0],'dif_num_indice_pre_2'],x.loc[date,'dif_num_indice_pre_2']]
            
    else:
        print('Erro de tipo diferente de "selic" ou "pre"') 
    # date=datetime.datetime.strptime(date,'%Y-%m-%d').date()
    # x=x[x.index==x.index[0] | x.index.date()==date]
    # dates=x.index
    # amounts=x['dif_num_indice']
    t_aa=xirr(dates, amounts)
    t_am=(1+t_aa)**(1/12)-1
    # print('TII a.a.(%): ',t_aa*100)
    # print('TII a.m.(%): ',t_am*100)
    # print('Soma dos valores: ',-x['Valor'].sum())
    # print(dates)
    # print(amounts)
    return (t_aa*100,t_am*100,dates,amounts)


In [28]:
for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],'juros_real_acum_from_now']=cal_irr(df_juros_real,df_juros_real.index[i],'selic')[0]
    df_juros_real.loc[df_juros_real.index[i],'juros_real_am']=cal_irr(df_juros_real,df_juros_real.index[i],'selic')[1]

df_juros_real.head(30)

,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am
2023-01-28,100.00,100.00,100.00,0.00,0.00
2023-01-29,100.04,100.02,100.02,7.05,0.57
2023-01-30,100.07,100.03,100.04,7.06,0.57
2023-01-31,100.11,100.05,100.06,7.06,0.57
2023-02-01,100.14,100.07,100.07,6.41,0.52
2023-02-02,100.18,100.10,100.08,6.03,0.49
2023-02-03,100.21,100.12,100.09,5.77,0.47
2023-02-04,100.25,100.14,100.10,5.59,0.45
2023-02-05,100.28,100.17,100.12,5.46,0.44
2023-02-06,100.32,100.19,100.13,5.35,0.44


In [29]:
for i in range(0,len(df_juros_real)):
    # Primeiro dia do mês
    inicio_mes=df_juros_real.index[i]-timedelta(df_juros_real.index[i].day-1)
    if inicio_mes<=df_juros_real.index[0]:
        inicio_mes=df_juros_real.index[0]
    # print(f'Início do mês: {inicio_mes} - até: {df_juros_real.index[i]}')
    # Slice do DF do primeiro dia do mês até o dia da análise
    try:
        df_juros_real_i=df_juros_real[df_juros_real.index>=inicio_mes]
        df_juros_real_i=df_juros_real_i[df_juros_real_i.index<=df_juros_real.index[i]]
        # print(df_juros_real_i)
        df_juros_real.loc[df_juros_real.index[i],'juros_real_mesal']=cal_irr(df_juros_real_i,df_juros_real.index[i],'selic')[0]
    except:
        continue
df_juros_real.head(30)

,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am,juros_real_mesal
2023-01-28,100.00,100.00,100.00,0.00,0.00,0.00
2023-01-29,100.04,100.02,100.02,7.05,0.57,7.05
2023-01-30,100.07,100.03,100.04,7.06,0.57,7.06
2023-01-31,100.11,100.05,100.06,7.06,0.57,7.06
2023-02-01,100.14,100.07,100.07,6.41,0.52,0.00
2023-02-02,100.18,100.10,100.08,6.03,0.49,4.50
2023-02-03,100.21,100.12,100.09,5.77,0.47,4.51
2023-02-04,100.25,100.14,100.10,5.59,0.45,4.51
2023-02-05,100.28,100.17,100.12,5.46,0.44,4.51
2023-02-06,100.32,100.19,100.13,5.35,0.44,4.51


In [30]:
def calc_pre_periodo(data_f,taxa_pre):
    n_dias=(data_f-datetime.datetime.today().date()).days
    return 100*(1+taxa_pre/100)**(n_dias/365)

In [31]:
calc_pre_periodo(datetime.datetime(2022,10,29).date(),12)

97.2140877874001

In [35]:
# Gera num_indice_taxa_pre-fixa
taxa_pre_1=12.85
taxa_pre_2=13.19
for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],f'num_indice_pre_1']=calc_pre_periodo(df_juros_real.index[i].date(),taxa_pre_1)
    df_juros_real.loc[df_juros_real.index[i],f'num_indice_pre_2']=calc_pre_periodo(df_juros_real.index[i].date(),taxa_pre_2)

df_juros_real


,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am,juros_real_mesal,num_indice_pre_1,num_indice_pre_2,dif_num_indice_pre_1,dif_num_indice_pre_2,juros_real_acum_from_now_pre_14.07%,juros_real_acum_from_now_pre_13.5%
2023-01-28,100.00,100.00,100.00,0.00,0.00,0.00,100.00,100.00,100.00,100.00,0.00,0.00
2023-01-29,100.04,100.02,100.02,7.05,0.57,7.05,100.03,100.03,100.02,100.02,7.36,6.82
2023-01-30,100.07,100.03,100.04,7.06,0.57,7.06,100.07,100.07,100.04,100.04,7.36,6.82
2023-01-31,100.11,100.05,100.06,7.06,0.57,7.06,100.10,100.10,100.06,100.05,7.36,6.82
2023-02-01,100.14,100.07,100.07,6.41,0.52,0.00,100.13,100.14,100.07,100.07,6.71,6.18
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-11,124.13,108.66,115.47,7.99,0.64,4.60,125.38,126.09,119.27,118.08,9.88,9.29
2024-12-12,124.16,108.68,115.48,7.98,0.64,4.60,125.43,126.13,119.30,118.11,9.88,9.29
2024-12-13,124.19,108.69,115.49,7.98,0.64,4.60,125.47,126.18,119.33,118.13,9.87,9.29
2024-12-14,124.22,108.71,115.51,7.97,0.64,4.60,125.51,126.22,119.36,118.16,9.87,9.28


In [36]:
# Agora que tenho num_indice_pre -> realizando os cálculos análogamente ao num_indice_selic
df_juros_real['dif_num_indice_pre_1']=df_juros_real.apply(lambda x: x.num_indice_pre_1-x.num_indice_ipca+100,axis=1)
df_juros_real['dif_num_indice_pre_2']=df_juros_real.apply(lambda x: x.num_indice_pre_2-x.num_indice_ipca+100,axis=1)


for i in range(0,len(df_juros_real)):
    df_juros_real.loc[df_juros_real.index[i],f'juros_real_acum_from_now_pre_{taxa_pre_1}%']=cal_irr(df_juros_real,df_juros_real.index[i],'pre_1')[0]
    df_juros_real.loc[df_juros_real.index[i],f'juros_real_acum_from_now_pre_{taxa_pre_2}%']=cal_irr(df_juros_real,df_juros_real.index[i],'pre_2')[0]

df_juros_real

,num_indice_selic,num_indice_ipca,dif_num_indice,juros_real_acum_from_now,juros_real_am,juros_real_mesal,num_indice_pre_1,num_indice_pre_2,dif_num_indice_pre_1,dif_num_indice_pre_2,juros_real_acum_from_now_pre_14.07%,juros_real_acum_from_now_pre_13.5%,juros_real_acum_from_now_pre_12.85%,juros_real_acum_from_now_pre_13.19%
2023-01-28,100.00,100.00,100.00,0.00,0.00,0.00,100.00,100.00,100.00,100.00,0.00,0.00,0.00,0.00
2023-01-29,100.04,100.02,100.02,7.05,0.57,7.05,100.03,100.03,100.02,100.02,7.36,6.82,6.21,6.53
2023-01-30,100.07,100.03,100.04,7.06,0.57,7.06,100.07,100.07,100.03,100.03,7.36,6.82,6.21,6.53
2023-01-31,100.11,100.05,100.06,7.06,0.57,7.06,100.10,100.10,100.05,100.05,7.36,6.82,6.21,6.53
2023-02-01,100.14,100.07,100.07,6.41,0.52,0.00,100.13,100.14,100.06,100.06,6.71,6.18,5.57,5.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-11,124.13,108.66,115.47,7.99,0.64,4.60,125.38,126.09,116.72,117.43,9.88,9.29,8.62,8.97
2024-12-12,124.16,108.68,115.48,7.98,0.64,4.60,125.43,126.13,116.75,117.46,9.88,9.29,8.61,8.97
2024-12-13,124.19,108.69,115.49,7.98,0.64,4.60,125.47,126.18,116.77,117.48,9.87,9.29,8.61,8.96
2024-12-14,124.22,108.71,115.51,7.97,0.64,4.60,125.51,126.22,116.80,117.51,9.87,9.28,8.61,8.96


In [37]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_juros_real,x=df_juros_real.index,y=['juros_real_acum_from_now','juros_real_mesal',f'juros_real_acum_from_now_pre_{taxa_pre_1}%',f'juros_real_acum_from_now_pre_{taxa_pre_2}%'])
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/Juros_real_forcast.html', auto_open=False)
fig.show()
